In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


In [2]:
df_sample = pd.read_csv("/kaggle/input/playground-series-s5e7/sample_submission.csv")
df_train = pd.read_csv("/kaggle/input/playground-series-s5e7/train.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s5e7/test.csv")

In [3]:
columns = df_train.columns[1:-1]
modes = df_train.drop(columns=['id', 'Personality']).mode().iloc[0]
df_train[columns] = df_train[columns].fillna(modes)

In [4]:
def to_numeric(df):
    df['Time_spent_Alone'] = df['Time_spent_Alone'].astype(int)
    df['Stage_fear'] = df['Stage_fear'].map({'Yes': 1, 'No': 0})
    df['Social_event_attendance'] = df['Social_event_attendance'].astype(int)
    df['Going_outside'] = df['Going_outside'].astype(int)
    df['Drained_after_socializing'] = df['Drained_after_socializing'].map({'Yes': 1, 'No': 0})
    df['Friends_circle_size'] = df['Friends_circle_size'].astype(int)
    df['Post_frequency'] = df['Post_frequency'].astype(int)
    return df

df_train = to_numeric(df_train)
df_train['Personality'] = df_train['Personality'].map({'Introvert': 1, 'Extrovert': 0})

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier

scaler = StandardScaler()

X = df_train.drop(columns=['id', 'Personality']).values
y = df_train['Personality'].values

gbc = GradientBoostingClassifier(
    learning_rate=0.01,
    max_depth=3,
    max_features='sqrt',
    min_samples_split=2,
    n_estimators=200,
    subsample=0.8,
    random_state=42
)

gbc.fit(scaler.fit_transform(X), y)

GradientBoostingClassifier(learning_rate=0.01, max_features='sqrt',
                           n_estimators=200, random_state=42, subsample=0.8)

In [6]:
X_test = df_test.drop(columns=['id'])

X_test = X_test.fillna(modes)
X_test = to_numeric(X_test)

X_test = X_test.values
X_test_scaled = scaler.transform(X_test)

y_pred = gbc.predict(X_test_scaled)
df_sample['Personality'] = pd.Series(y_pred).map({1: "Introvert", 0: "Extrovert"})

df_sample.to_csv('submission.csv', index=False)